In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [30]:
image_size = (256, 256)
batch_size = 32

In [31]:
datagen = ImageDataGenerator(rescale=1.0/255.0)

In [32]:
train_generator = datagen.flow_from_directory(
    './Class/',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 120 images belonging to 4 classes.


In [33]:
test_generator = datagen.flow_from_directory(
    "./FaceImages/",
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

Found 4 images belonging to 1 classes.


In [34]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

In [35]:
base_model.trainable = False

In [36]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

In [37]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=500
)

Epoch 1/500
3/3 [==============================] - 2s 219ms/step - loss: 1.6725 - accuracy: 0.2727
Epoch 2/500
3/3 [==============================] - 1s 244ms/step - loss: 1.3120 - accuracy: 0.4318
Epoch 3/500
3/3 [==============================] - 1s 201ms/step - loss: 1.3485 - accuracy: 0.4545
Epoch 4/500
3/3 [==============================] - 1s 258ms/step - loss: 1.3019 - accuracy: 0.4318
Epoch 5/500
3/3 [==============================] - 1s 224ms/step - loss: 1.3208 - accuracy: 0.3636
Epoch 6/500
3/3 [==============================] - 1s 243ms/step - loss: 0.7925 - accuracy: 0.6771
Epoch 7/500
3/3 [==============================] - 1s 210ms/step - loss: 0.9944 - accuracy: 0.5729
Epoch 8/500
3/3 [==============================] - 1s 200ms/step - loss: 0.6785 - accuracy: 0.6979
Epoch 9/500
3/3 [==============================] - 1s 184ms/step - loss: 0.6266 - accuracy: 0.7273
Epoch 10/500
3/3 [==============================] - 1s 200ms/step - loss: 0.6190 - accuracy: 0.7841
Epoch 11/

In [39]:
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

1/1 [==============================] - 1s 598ms/step


In [40]:
class_labels = train_generator.class_indices
class_labels = dict((v, k) for k, v in class_labels.items())


In [41]:
predicted_labels = [class_labels[prediction] for prediction in predicted_classes]
print(predicted_labels)

['ChrisEvans', 'Hemsworth', 'Renner', 'RDJ']


In [42]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])